In [2]:
# train a Deep Learning model on MNIST Data to recognize digits in images using pytorch
# and compute the carbon footprint of the training process

In [3]:
# import libraries
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

from codecarbon import EmissionsTracker

In [4]:
# Download training data from open datasets.
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)
training_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [5]:
# Download test data from open datasets.
test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [6]:
# set the batch size
batch_size = 64

In [7]:
# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [8]:
# creating model
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using mps device


In [9]:
# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [10]:
# optimizer and loss function
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [11]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [12]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [13]:
# import the tracker using explicit mode
tracker = EmissionsTracker()
tracker.start()

epochs = 100
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

emissions: float = tracker.stop()
print(emissions)

[codecarbon INFO @ 19:23:16] [setup] RAM Tracking...
[codecarbon INFO @ 19:23:16] [setup] GPU Tracking...
[codecarbon INFO @ 19:23:16] No GPU found.
[codecarbon INFO @ 19:23:16] [setup] CPU Tracking...
[codecarbon WARNING @ 19:23:16] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 19:23:17] CPU Model on constant consumption mode: Apple M1
[codecarbon INFO @ 19:23:17] >>> Tracker's metadata:
[codecarbon INFO @ 19:23:17]   Platform system: macOS-14.0-arm64-arm-64bit
[codecarbon INFO @ 19:23:17]   Python version: 3.8.18
[codecarbon INFO @ 19:23:17]   CodeCarbon version: 2.3.1
[codecarbon INFO @ 19:23:17]   Available RAM : 16.000 GB
[codecarbon INFO @ 19:23:17]   CPU count: 8
[codecarbon INFO @ 19:23:17]   CPU model: Apple M1
[codecarbon INFO @ 19:23:17]   GPU count: None
[codecarbon INFO @ 19:23:17]   GPU model: None


Epoch 1
-------------------------------
loss: 2.303898  [   64/60000]
loss: 2.293610  [ 6464/60000]
loss: 2.296293  [12864/60000]
loss: 2.284387  [19264/60000]
loss: 2.289701  [25664/60000]
loss: 2.282427  [32064/60000]
loss: 2.269239  [38464/60000]
loss: 2.284912  [44864/60000]
loss: 2.264404  [51264/60000]
loss: 2.246340  [57664/60000]
Test Error: 
 Accuracy: 42.3%, Avg loss: 2.256707 

Epoch 2
-------------------------------
loss: 2.257491  [   64/60000]
loss: 2.245530  [ 6464/60000]
loss: 2.257179  [12864/60000]
loss: 2.224052  [19264/60000]
loss: 2.238951  [25664/60000]
loss: 2.230871  [32064/60000]
loss: 2.208492  [38464/60000]
loss: 2.241544  [44864/60000]
loss: 2.201428  [51264/60000]
loss: 2.178110  [57664/60000]
Test Error: 
 Accuracy: 59.4%, Avg loss: 2.190744 

Epoch 3
-------------------------------
loss: 2.192082  [   64/60000]
loss: 2.174016  [ 6464/60000]
loss: 2.197155  [12864/60000]
loss: 2.133804  [19264/60000]
loss: 2.160720  [25664/60000]
loss: 2.148463  [32064/600

[codecarbon INFO @ 19:23:33] Energy consumed for RAM : 0.000025 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 19:23:33] Energy consumed for all CPUs : 0.000021 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 19:23:33] 0.000046 kWh of electricity used since the beginning.


loss: 2.050756  [ 6464/60000]
loss: 2.090658  [12864/60000]
loss: 1.982455  [19264/60000]
loss: 2.024094  [25664/60000]
loss: 2.002959  [32064/60000]
loss: 1.932582  [38464/60000]
loss: 2.027435  [44864/60000]
loss: 1.919657  [51264/60000]
loss: 1.865726  [57664/60000]
Test Error: 
 Accuracy: 71.7%, Avg loss: 1.883544 

Epoch 5
-------------------------------
loss: 1.896740  [   64/60000]
loss: 1.838443  [ 6464/60000]
loss: 1.899429  [12864/60000]
loss: 1.737606  [19264/60000]
loss: 1.788625  [25664/60000]
loss: 1.755551  [32064/60000]
loss: 1.659450  [38464/60000]
loss: 1.796546  [44864/60000]
loss: 1.647550  [51264/60000]
loss: 1.575996  [57664/60000]
Test Error: 
 Accuracy: 74.8%, Avg loss: 1.587558 

Epoch 6
-------------------------------
loss: 1.622177  [   64/60000]
loss: 1.525411  [ 6464/60000]
loss: 1.608655  [12864/60000]
loss: 1.412227  [19264/60000]
loss: 1.459377  [25664/60000]
loss: 1.419737  [32064/60000]
loss: 1.326331  [38464/60000]
loss: 1.494323  [44864/60000]
loss: 

[codecarbon INFO @ 19:23:48] Energy consumed for RAM : 0.000050 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 19:23:48] Energy consumed for all CPUs : 0.000042 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 19:23:48] 0.000092 kWh of electricity used since the beginning.


...

loss: 0.200786  [ 6464/60000]
loss: 0.130489  [12864/60000]
loss: 0.248671  [19264/60000]
loss: 0.138110  [25664/60000]
loss: 0.241037  [32064/60000]
loss: 0.133118  [38464/60000]
loss: 0.301080  [44864/60000]
loss: 0.170701  [51264/60000]
loss: 0.311203  [57664/60000]
Test Error: 
 Accuracy: 94.0%, Avg loss: 0.205632 

Epoch 97
-------------------------------
loss: 0.133362  [   64/60000]
loss: 0.199787  [ 6464/60000]
loss: 0.129708  [12864/60000]
loss: 0.247244  [19264/60000]
loss: 0.136980  [25664/60000]
loss: 0.240194  [32064/60000]
loss: 0.132159  [38464/60000]
loss: 0.299894  [44864/60000]
loss: 0.169486  [51264/60000]
loss: 0.309743  [57664/60000]
Test Error: 
 Accuracy: 94.0%, Avg loss: 0.204467 

Epoch 98
-------------------------------
loss: 0.132205  [   64/60000]
loss: 0.198797  [ 6464/60000]
loss: 0.128905  [12864/60000]
loss: 0.245820  [19264/60000]
loss: 0.135854  [25664/60000]
loss: 0.239337  [32064/60000]
loss: 0.131171  [38464/60000]
loss: 0.298702  [44864/60000]
loss

[codecarbon INFO @ 19:30:18] Energy consumed for RAM : 0.000700 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 19:30:18] Energy consumed for all CPUs : 0.000584 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 19:30:18] 0.001284 kWh of electricity used since the beginning.


loss: 0.130205  [38464/60000]
loss: 0.297534  [44864/60000]
loss: 0.167182  [51264/60000]
loss: 0.306891  [57664/60000]
Test Error: 
 Accuracy: 94.1%, Avg loss: 0.202175 

Epoch 100
-------------------------------
loss: 0.129946  [   64/60000]
loss: 0.196821  [ 6464/60000]
loss: 0.127277  [12864/60000]
loss: 0.242976  [19264/60000]
loss: 0.133659  [25664/60000]
loss: 0.237625  [32064/60000]
loss: 0.129258  [38464/60000]
loss: 0.296395  [44864/60000]
loss: 0.166006  [51264/60000]
loss: 0.305475  [57664/60000]


[codecarbon INFO @ 19:30:24] Energy consumed for RAM : 0.000710 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 19:30:24] Energy consumed for all CPUs : 0.000592 kWh. Total CPU Power : 5.0 W
[codecarbon INFO @ 19:30:24] 0.001302 kWh of electricity used since the beginning.


Test Error: 
 Accuracy: 94.1%, Avg loss: 0.201046 

Done!
0.0004436852867188026
